# Interactive Cellpose 3D training workflow

### Upsampling Z dimension
Check how the image would look when Z sampling is rescaled to be isotropic

In [ ]:
import imageio
from skimage.transform import rescale
import napari

In [ ]:
# Load a 3D image
img_path = '/Users/joel/Desktop/object_8/220508_215839_B03_T0001F001L01A04Z01C01O00008_TIF-OVR.tif'
img = imageio.volread(img_path)

In [ ]:
# Upscale the Z dimension to be isotropic
anisotropy = 2.7
img_rescaled = rescale(img, scale=(anisotropy, 1.0, 1.0))

In [ ]:
# Check the image in the viewer
viewer = napari.Viewer()
viewer.add_image(img)
viewer.add_image(img_rescaled)

In [ ]:
# Save rescaled image
output_path = '/Users/joel/Desktop/object_8/220508_215839_C01_Rescaled.tif'
imageio.volwrite(output_path, img_rescaled)

# Annotation & training workflow

Cellpose is trained on 2D slices. Thus, we generate an isotropic 3D image by upsampling the same way that cellpose uses internally. Then, we run cellpose on this 3D image and pick some planes. We correct these planes and feed them into model training.
Iteratively run this workflow until the model is good enough.

1. Run Cellpose with a given parameters  
    a) With a default model or a custom one
2. Browse the result in napari, check performance
    a) optionally check the flows to see why the performance suffered
3. Get a few planes you want to retrain the model on (choose e.g. 3 xy, 3 xz or yz planes)  
    a) Correct the labels for these planes. You can check the other planes to ensure you feed in correct training data
    b) Save those planes to the train_dir / test_dir folder. Add some to the test set to evaluate whether the training has a generalizable effect
4. Run a retraining on those images
5. Then apply the network to a new 3D organoid => start again at 1)
    a) Alternatively, apply it back to the same 3D data and correct different planes
    
Training workflow is based on https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/run_cellpose_2.ipynb#scrollTo=gCcbs722BYd0

In [ ]:
from cellpose import models, io
import imageio
from skimage.transform import rescale
import napari
from pathlib import Path
import os
import glob

### 1. Run a cellpose model on some 3D images

In [ ]:
# Load the image
img_path = '/Users/joel/Desktop/object_8/220508_215839_B03_T0001F001L01A04Z01C01O00008_TIF-OVR.tif'
img = imageio.volread(img_path)
# Important: Set anisotropy correctly based on the Z spacing!
anisotropy = 2.7

# Should you correct anisotropy in the image itself, before passing it to cellpose?
# Useful to generate isotropic data, the same as the cellpose model does internally. 
# Thus, useful when we want to generate new training cases. 
# But not what we want to look at eventaully (when predicting on the real data)
correct_anisotropy = True
if correct_anisotropy:
    img = rescale(img, scale=(anisotropy, 1.0, 1.0))
    anisotropy = 1.0

In [ ]:
# Helper function to run the cellpose model
def run_cellpose_model(
                       img,
                       do_3D = True,
                       anisotropy = 2.7,
                       model = 'cyto2',
                       pretrained_model = None,
                       cellprob_th = 0.0,
                       diameter = 35,
                       channels=[0,0],
                       use_GPU = True
                      ):
    logger = io.logger_setup()
    # Run cellpose model inference
    if pretrained_model:
        model = models.CellposeModel(gpu=use_GPU, pretrained_model=pretrained_model)
    else:
        model = models.CellposeModel(gpu=use_GPU, model_type=model)
    mask, flows, styles = model.eval(img, 
                                            channels=channels, 
                                            diameter=diameter, 
                                            anisotropy=anisotropy, 
                                            do_3D=do_3D, 
                                            #net_avg=False, 
                                            #augment=False, 
                                            cellprob_threshold=cellprob_th)
        
    return mask, flows, styles

In [ ]:
diameter = 25
model_type = "cyto2"
mask, flows, _ = run_cellpose_model(img, anisotropy=anisotropy, diameter=diameter, model=model_type)

# Use this approach to run a pretrained model later
#pretrained_model = '/Users/joel/Dropbox/Joel/FMI/Code/cellpose/models/retrained_cyto2'
#mask, flows, _ = run_cellpose_model(img, anisotropy=anisotropy, diameter=diameter, pretrained_model=pretrained_model)

### 2. Browse the result in napari

In [ ]:
viewer = napari.Viewer()
image_layer = viewer.add_image(img, scale = (anisotropy, 1, 1))
label_layer = viewer.add_labels(mask, scale = (anisotropy, 1, 1))
viewer.add_image(flows[0], scale = (anisotropy, 1, 1), name = 'Flows', visible=False)

# 3. Export corrected annotation planes
Make sure the labels are good for the planes you want to export as new training data

Then, enter the index of the plane in the training or test list to export it into that directory

In [ ]:
# Export individual planes (both image data and labels)
base_name = '220508_215839_B03_T0001F001L01A04Z01C01O00008'
train_dir = 'train_dir'
test_dir = 'test_dir'

xy_planes_train = [129]
xy_panes_test = [130]
xz_planes_train = []
xz_planes_test = []
yz_planes_train = []
yz_planes_test = []

training_lists = [xy_planes_train, xz_planes_train, yz_planes_train]
test_lists = [xy_panes_test, xz_planes_test, yz_planes_test]

for dimension, curr_list in enumerate(training_lists):
    save_new_annotations(
        plane_list = curr_list, 
        dimension = 0,
        save_dir = train_dir,
        base_name = base_name,
    )

for dimension, curr_list in enumerate(test_lists):
    save_new_annotations(
        plane_list = curr_list, 
        dimension = 0,
        save_dir = test_dir,
        base_name = base_name,
    )

In [ ]:
def save_img(img, base_name, plane, dir_path):
    imageio.imwrite(Path(dir_path) / f'{base_name}_{plane}.tif', img)
    
def save_label(label_img, base_name, plane, dir_path):
    # TODO: Do those need to be .npy pickle files or does it accept tifs?
    imageio.imwrite(Path(dir_path) / f'{base_name}_{plane}_seg.tif', label_img)

def save_new_annotations(plane_list, dimension, save_dir, base_name):
    for plane in plane_list:
        if dimension == 0:
            img_plane = image_layer.data[plane, :, :]
            label_plane = label_layer.data[plane, :, :]
        elif dimmension == 1:
            img_plane = image_layer.data[:, plane, :]
            label_plane = label_layer.data[:, plane, :]
        elif dimmension == 2:
            img_plane = image_layer.data[:, :, plane]
            label_plane = label_layer.data[:, :, plane]
        else:
            print('Not a valid dimension chosen')
            return
        plane_names = {0: 'xy', 1: 'xz', 2: 'yz'}
        new_base_name = base_name + '_' + plane_names[dimension]
        save_img(img_plane, base_name = new_base_name, plane = plane, dir_path = save_dir)
        save_label(label_plane, base_name = new_base_name, plane = plane, dir_path = save_dir)

# 4. Retrain the cellpose model
Currently always retraining from the base model. One could also iteratively retrain by having different training directories

In [ ]:
# Parameters to change
# Make this unique for every new network you train. Otherwise, it overwrites the existing network
model_name = "retrained_cyto2"

initial_model = 'cyto2'
model_dir = '/Users/joel/Dropbox/Joel/FMI/Code/cellpose/models'
# On Macs, using the GPU for training leads to 
# `RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn`
# If it works on your machine, set the use_GPU=True
use_GPU = False
# Works when a single input channel is used
channels = [0, 0]
n_epochs = 100

# Don't change these 2 parameters
weight_decay = 0.0001
learning_rate = 0.1

model = models.CellposeModel(gpu=use_GPU, model_type=initial_model)


In [ ]:
# Load train data & labels from disk
# FIXME: Hacky way, make this more robust
def load_train_data(train_dir, mask_filter = '_seg.tif'):
    files = glob.glob(str(Path(train_dir) / '*.tif'))
    train_paths = [f for f in files if not f.endswith(mask_filter)]
    label_paths = [f[:-4] + mask_filter for f in train_paths]
    train_imgs = []
    for train_p in train_paths:
        train_imgs.append(imageio.v2.imread(train_p))
    label_imgs = []
    for label_p in label_paths:
        label_imgs.append(imageio.v2.imread(label_p))
    return train_imgs, label_imgs


In [ ]:
train_dir = 'train_dir'
train_data, train_labels = load_train_data(train_dir)

test_dir = 'test_dir'
test_data, test_labels = load_train_data(test_dir)

In [ ]:
# If you have no test data, comment out the two lines with test_data & test_labels
logger = io.logger_setup()
new_model_path = model.train(train_data, train_labels, 
                              test_data=test_data,
                              test_labels=test_labels,
                              channels=channels, 
                              save_path=model_dir, 
                              n_epochs=n_epochs,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=8,
                              model_name=model_name)